In [181]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [182]:
csv_file_path = '/home/lucas/UFOP/ple_2020/analise_midias_sociais/final-work/data/all_pp_posts_bolsonaro_posts.csv'
df_candidate = pd.read_csv(csv_file_path,encoding = 'utf-8')

In [183]:
df_candidate.head(15)

,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
0,2020-03-22T18:47:55+0000,211857482296579_1826486634166981,agora videoconferencia presidente jair bolsona...,- agora por videoconferencia o presidente jair...,23151,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,129465,147,318,21,112,10345,True
1,2020-03-22T16:30:39+0000,211857482296579_1826339070848404,inscricoes selecao profissionais atuacao progr...,- as inscricoes para a selecao de 5.800 profis...,5703,mobile_status_update,None,52238,128,61,5,43,2514,True
2,2020-03-22T13:37:26+0000,211857482296579_1826130007535977,ministerio saude ministro mandetta reuniao vir...,- estamos no ministerio da saude com o ministr...,11818,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,97986,73,141,16,65,6292,True
3,2020-03-22T11:07:01+0000,211857482296579_1825937284221916,governo federal distribui milhoes testes rapid...,- o governo federal distribui milhoes de teste...,39284,mobile_status_update,None,165539,88,393,20,68,16747,True
4,2020-03-22T02:06:48+0000,211857482296579_1825422034273441,chama precaucao medicamento barato caso venha ...,- isso se chama precaucao . o medicamento e ba...,27236,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9052732...,162513,968,318,88,509,20723,True
5,2020-03-21T22:22:48+0000,211857482296579_1825061764309468,reconheco seriedade momento temor muitos brasi...,- reconheco a seriedade do momento e o temor d...,40794,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9065012...,305358,918,478,266,1267,48834,True
6,2020-03-21T18:36:01+0000,211857482296579_1824728371009474,hospital albert einstein possivel cura pacient...,- hospital albert einstein e a possivel cura d...,201803,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/75...,215237,480,815,60,431,34278,True
7,2020-03-21T11:33:30+0000,211857482296579_1824261207722857,dois dedos prosa ratinho assista link youtube,- dois dedos de prosa com ratinho . - assista ...,33971,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/83...,106117,336,317,113,624,15368,True
8,2020-03-21T02:37:00+0000,211857482296579_1823872341095077,assinei medida provisoria deixando claro ser c...,- assinei medida provisoria deixando claro ser...,9174,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/s720x72...,112873,181,200,106,1121,7582,True
9,2020-03-21T00:16:41+0000,211857482296579_1823743497774628,via eduardo bolsonaro,via eduardo bolsonaro,8150,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p600x60...,79762,435,111,13,160,6480,True


In [184]:
n_total_posts = len(df_candidate)
print('número total de posts: {0}'.format(str(n_total_posts)))

número total de posts: 3646


In [185]:
n_posts_with_textual_messages = df_candidate[df_candidate.has_textual_message == True].shape[0]
n_posts_without_textual_messages = df_candidate[df_candidate.has_textual_message == False].shape[0]

print('número de posts com mensagens textuais: {0}'.format(str(n_posts_with_textual_messages)))
print('número de posts sem mensagens textuais: {0}'.format(str(n_posts_without_textual_messages)))

número de posts com mensagens textuais: 3394
número de posts sem mensagens textuais: 252


In [180]:
# Devemos remover esses posts sem menssagem antes de continuar a nossa análise?
# renomear depois o df caso sim

#filter_mask  = df_candidate['has_textual_message'] == True
#df_candidate = df_candidate[filter_mask]
#print('número de posts do df desconsiderando os posts sem menssagens textuais: {0}'.format(str(len(df_candidate))))

número de posts do df desconsiderando os posts sem menssagens textuais: 3394


In [186]:
# separando o df de treinamento (30%) e o df de testes(70%) de maneira 'embaralhada'
df_training, df_test = train_test_split(df_candidate,train_size=0.7,test_size=0.3,shuffle=True)

In [191]:
print('número de posts do conjunto de treinamento (70%): {0}'.format(str(len(df_training))))
df_training.head(5)

número de posts do conjunto de treinamento (70%): 2552


,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
2656,2016-06-12T13:11:00+0000,211857482296579_642906735858316,porque nao existe turismo brasil,porque nao existe turismo no brasil .,13285,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/12...,32748,159,92,15,13,637,True
78,2020-03-15T15:46:59+0000,211857482296579_1814654165350228,belo horizontemg,belo horizonte-mg :,21564,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/75...,53782,129,252,14,70,6982,True
535,2019-12-05T12:57:51+0000,211857482296579_1692694834212829,bento goncalvesrs,bento goncalves/rs,11289,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/73...,58942,188,498,14,49,6853,True
3192,2014-12-16T16:47:32+0000,211857482296579_415518068597185,diplomado setima vez desta deputado federal vo...,diplomado pela setima vez desta como deputado ...,28962,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/1510925...,127223,10,2,0,0,88,True
161,2020-03-02T09:53:44+0000,211857482296579_1799463033536008,rodada venda bens so trazem gastos pagador imp...,mais uma rodada de venda de bens que so trazem...,4698,mobile_status_update,None,67454,54,118,5,19,4455,True


In [192]:
print('número de posts do conjunto de teste (30%): {0}'.format(str(len(df_test))))
df_test.head(5)

número de posts do conjunto de teste (30%): 1094


,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
2964,2015-10-06T18:45:45+0000,211857482296579_543676842447973,instituto parana pesquisas divulga nova pesqui...,o instituto parana pesquisas divulga nova pesq...,3998,added_photos,https://scontent.xx.fbcdn.net/v/t31.0-8/s720x7...,22108,0,0,0,0,0,True
2615,2016-06-23T21:40:00+0000,211857482296579_647995242016132,todos brasil obrigado,somos todos brasil ! obrigado !,2278,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/13...,22174,39,12,3,2,406,True
2433,2016-09-24T20:34:44+0000,211857482296579_694242714058051,caminhada campo grande rj obrigado rio janeiro...,caminhada campo grande / rj . obrigado rio de ...,717,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/14...,7506,40,7,1,5,262,True
462,2020-01-08T22:42:30+0000,211857482296579_1738096563005989,reduzir preco combustiveis quase produto final...,- como reduzir o preco dos combustiveis ? - qu...,47941,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/72...,101561,435,774,129,119,5628,True
688,2019-09-26T23:16:45+0000,211857482296579_1602440679904912,pastor silas malafaia diz governo bolsonaro na...,- pastor silas malafaia diz para quem o govern...,63148,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/70...,86347,1029,715,30,173,8468,True


In [ ]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [53]:
n_top_words = 5

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf = tfidf_vectorizer.fit_transform(df_candidate['pre_processed_message'])

nmf = NMF(n_components=200, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)

tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

Topic #0: zuero eficiente eeaer eear efeito
Topic #1: obrigado apoio fortaleza pernambuco paraiba
Topic #2: tudo acima paraquedista brigada copacabanarj
Topic #3: link youtube governo sobre politica
Topic #4: zuero eficiente eeaer eear efeito
Topic #5: boa noite todos segundafeira vista
Topic #6: rio janeiro maratona hoje junho
Topic #7: acima deus todos brasil tudo
Topic #8: semana live presidente quintafeira assuntos
Topic #9: parte roraima final bahia paraiba
Topic #10: bolsonaro carlos flavio eduardo vereador
Topic #11: paulo sao folha santos sargentos
Topic #12: brasilia agora hmin df compareca
Topic #13: bom todos dia domingo fim
Topic #14: zuero eficiente eeaer eear efeito
Topic #15: abraco forte todos horizonte belo
Topic #16: zuero eficiente eeaer eear efeito
Topic #17: zuero eficiente eeaer eear efeito
Topic #18: sp praia grande grandesp mogi
Topic #19: zuero eficiente eeaer eear efeito
Topic #20: povo brasileiro respeito carinho pais
Topic #21: grande sul rio dia campo
Topic

In [ ]:
cv = CountVectorizer(max_df = 0.8, min_df = 2)
df = cv.fit_transform(df_candidate['pre_processed_message'])

In [20]:
model_lda = LatentDirichletAllocation(n_components = 10)
model_lda.fit(df)

LatentDirichletAllocation()

In [14]:
for index, topic in enumerate(model_lda.components_):
    print(f'Top 10 words for #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for #0
['brasil', 'milit', 'alun', 'federal', 'public', 'cas', 'ensin', 'program', 'educaca', 'escol']


Top 10 words for #1
['pod', 'pov', 'pt', 'vid', 'polit', 'tud', 'acim', 'tod', 'nao', 'brasil']


Top 10 words for #2
['cam', 'deput', 'glob', 'direit', 'sobr', 'jornal', 'brasil', 'verdad', 'bolsonar', 'nao']


Top 10 words for #3
['part', 'hoj', 'milit', 'tod', 'janeir', 'grand', 'abrac', 'rio', 'brasil', 'obrig']


Top 10 words for #4
['sao', 'seguranc', 'econom', 'nao', 'brasil', 'med', 'public', 'ministr', 'govern', 'ministeri']


Top 10 words for #5
['infraestrutur', 'saud', 'nov', 'ministr', 'milho', 'estad', 'econom', 'govern', 'brasil', 'ministeri']


Top 10 words for #6
['tod', 'ser', 'dilm', 'president', 'eduard', 'sao', 'jair', 'deput', 'nao', 'bolsonar']


Top 10 words for #7
['agor', 'flavi', 'sobr', 'eduard', 'brasil', 'tod', 'youtub', 'link', 'jair', 'bolsonar']


Top 10 words for #8
['jair', 'dilm', 'sobr', 'contr', 'deput', 'lei', 'brasil', 'nao', 'pt'

In [19]:
feature_names = cv.get_feature_names()
number_top_words = 10
for topic_idx, topic in enumerate(model_lda.components_):
    message = "Topic {0}: ".format(str(topic_idx))
    message += " ".join([feature_names[i]
                         for i in topic.argsort()[:-number_top_words - 1:-1]])
    print(message)
print()

Topic 0: brasil todos nao acima tudo governo deus sao pais pt
Topic 1: governo ministerio brasil programa bilhoes milhoes anos federal ano economia
Topic 2: sao pt nao paulo verdade sobre jornal comissao camara so
Topic 3: bolsonaro rio nao carlos janeiro brasil ministerio dilma obrigado flavio
Topic 4: bolsonaro jair youtube link brasil sobre eduardo entrevista obrigado presidente
Topic 5: bolsonaro federal nao eduardo lei brasil deputado sp canal sao
Topic 6: todos voto nao impresso bolsonaro hoje abraco brasil povo obrigado
Topic 7: nao bolsonaro educacao direitos militar humanos brasil sobre escolas rio
Topic 8: nao todos brasil presidente ser ministro bolsonaro boa apoio estado
Topic 9: obrigado nao pt contra brasil publica psol crime grande ministerio

